Author：AbsoluteX
+ E-mail：118020043@link.cuhk.edu.cn / l276660317@gmail.com
+ start-date: 2022-04-13

/----- 项目说明 -----/
+ 用于测试中文打分库的表现，尽可能多地网罗library并打分。将打分打印出来用肉眼校验可靠性
+ 所有的情绪打分位于0-1之间，0代表negative，1代表positive
+ ipynb文件运行需伴随附件utils_selfdefined.py文件，否则enrie无法运行。
+ 测试全部基于服务器仅cpu环境测试

输出的打分文件在路径下的csv内，请自取判断

In [1]:
import datetime
import re
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import pickle
sns.set_style('darkgrid')
import joblib
import sys

import matplotlib.ticker as ticker

# 1.测试样本数据展示

In [2]:
data_path = '/dat/cqdata/socialmedia/eastmoney/guba/daily/'
file_list = os.listdir(data_path)
txt_file_list = list()
for i in file_list:
    if len(i)==8:
        if int(i[:4])>=2018:
            txt_file_list.append(i)
print("process file num is:",len(txt_file_list))
txt_file_list.sort()
print(txt_file_list[:10])

process file num is: 1558
['20180101', '20180102', '20180103', '20180104', '20180105', '20180106', '20180107', '20180108', '20180109', '20180110']


In [3]:
def get_opinion_df(file_path):
    content = list()
    with open(file_path,"r",encoding="utf-8") as f:
        for lines in f.readlines():
            content.append(lines[:-1].split('|'))
    sentense = list()
    original_content = list()
    for i in content:
        if len(i)==6:
            sentense.append(i[-1])
            original_content.append(i)
        if len(i)>6:
            concat_sen = ''
            for j in i[5:]:
                concat_sen = concat_sen+ str(j)
            sentense.append(j)
            original_content.append(i)

    last_change = list()
    build_day = list()
    up_opinion = list()
    comments_num = list()
    stock_code = list()
    user_id = list()
    for i in original_content:
        try:
            last_change.append(i[0])
            build_day.append(i[1])
            up_opinion.append(i[2])
            comments_num.append(i[3])
            stock_code.append((i[4].split(','))[0])
            user_id.append((i[4].split(','))[1])
        except:
            sentense.pop(original_content.index(i))

    pos_sentense_day = dict()
    pos_sentense_day['last_change'] = last_change
    pos_sentense_day['stock_code'] = stock_code
    pos_sentense_day['sentense'] = sentense
    return pd.DataFrame(pos_sentense_day)

In [4]:
## 选择这一天的原因是
## 1.随机想了一天
## 2.这一天是交易日，舆情数据充足
target_file = '20180406'
opinion_df = get_opinion_df(data_path+target_file)
opinion_df.to_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv',
                  index=False,
                  encoding='utf_8_sig')
opinion_df.head()

,last_change,stock_code,sentense
0,20180406 23:52,600000,浦发为什么这么烂？
1,20180406 23:40,600000,反盗版
2,20180406 23:29,600000,最近十年搬起石头砸自己的脚”的有: 美国、日本
3,20180406 23:21,600000,88见，坐等数票子～
4,20180406 23:17,600000,请问这样的股市正常吗？


# 2.library的安装与展示

其中2.2 & 2.5 来自网站 paper with code

## 2.1 cemotion
+ 地址：https://github.com/Cyberbolt/Cemotion
+ 最近一次更新：14 months ago
+ 语言要求：python3
+ 安装：pip install cemotion
+ return：score in [0,1]

In [5]:
#使用列表进行批量分析
from cemotion import Cemotion

list_text = ['内饰蛮年轻的，而且看上去质感都蛮好，貌似本田所有车都有点相似，满高档的！',
'总而言之，是一家不会再去的店。']

c = Cemotion()
print(c.predict(list_text))

[['内饰蛮年轻的，而且看上去质感都蛮好，貌似本田所有车都有点相似，满高档的！', 0.999979], ['总而言之，是一家不会再去的店。', 0.064269]]


## 2.2 transformers
+ 地址：https://github.com/huggingface/transformers
+ 最近一次更新：6 days ago
+ 语言要求：Python 3.6+
+ 安装：pip install transformers
+ return：{'label': 'POSITIVE', 'score': 0.9996980428695679}这里的答案“正面” (positive) 具有 99 的置信度。

In [6]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
classifier(['内饰蛮年轻的，而且看上去质感都蛮好，貌似本田所有车都有点相似，满高档的！',
'总而言之，是一家不会再去的店。'])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'NEGATIVE', 'score': 0.6222265362739563},
 {'label': 'NEGATIVE', 'score': 0.9170925617218018}]

## 2.3 Jiagu
+ 地址：https://github.com/ownthink/Jiagu
+ 最近一次更新：2 years ago
+ 语言要求：Python3
+ 安装：pip install -U jiagu
+ return：('negative', 0.9957030885091285)

In [7]:
import jiagu
print(jiagu.sentiment('内饰蛮年轻的，而且看上去质感都蛮好，貌似本田所有车都有点相似，满高档的！'))
print(jiagu.sentiment('总而言之，是一家不会再去的店。'))

('negative', 0.934012968300321)
('negative', 0.9840209818894707)


## 2.4 cnsenti
+ 地址：https://github.com/hiDaDeng/cnsenti
+ 最近一次更新：11 months ago
+ 语言要求：Python3
+ 安装：pip install cnsenti
+ return：{'words': 10, 'sentences': 1, 'pos': 0, 'neg': 1}

In [8]:
from cnsenti import Sentiment

senti = Sentiment()
print(senti.sentiment_count('内饰蛮年轻的，而且看上去质感都蛮好，貌似本田所有车都有点相似，满高档的！'))
print(senti.sentiment_count('总而言之，是一家不会再去的店。'))

{'words': 24, 'sentences': 1, 'pos': 0, 'neg': 0}
{'words': 10, 'sentences': 1, 'pos': 0, 'neg': 1}


## 2.5 ernie
+ 服务提供方：百度
+ 持续更新
+ 安装：pip install paddlepaddle & pip install --upgrade paddlenlp
+ Ps：由于各种复现中的bug，pip过程混乱。不保证上述安装能够完成。若失败请联系笔者或在 https://aistudio.baidu.com/aistudio/projectdetail/3811233?forkThirdPart=1 网站下自行解决

In [9]:
import paddlenlp as ppnlp
from paddlenlp.datasets import load_dataset
from utils_selfdefined import predict

train_ds, dev_ds, test_ds = load_dataset(
    "chnsenticorp", splits=["train", "dev", "test"])

MODEL_NAME = "ernie-1.0"
tokenizer = ppnlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)
ernie_model = ppnlp.transformers.ErnieModel.from_pretrained(MODEL_NAME)
ernie_model = ppnlp.transformers.ErnieModel.from_pretrained(MODEL_NAME)
model = ppnlp.transformers.ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=len(train_ds.label_list))

batch_size = 32
data = [
    {"text":'这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'},
    {"text":'怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'},
    {"text":'作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'},
]
label_map = {0: 'negative', 1: 'positive'}

results = predict(
    model, data, tokenizer, label_map, batch_size=batch_size)
for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))
print(results)

[2022-04-15 17:24:30,726] [    INFO] - Already cached /home/cqintern08/.paddlenlp/models/ernie-1.0/vocab.txt
[2022-04-15 17:24:30,739] [    INFO] - Already cached /home/cqintern08/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-04-15 17:24:35,850] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']
[2022-04-15 17:24:36,177] [    INFO] - Already cached /home/cqintern08/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-04-15 17:24:41,207] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']
[2022-04-15 17:24:41,480] [    INFO] - Already cached /home/cqintern08/.paddlenlp/models/ernie-1.0/erni

Data: {'text': '这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'} 	 Lable: positive
Data: {'text': '怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'} 	 Lable: positive
Data: {'text': '作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'} 	 Lable: positive
['positive', 'positive', 'positive']


In [10]:
##predict_score is a self defined func for output score
from utils_selfdefined import predict_score
predict_score(model, data, tokenizer, label_map, batch_size=batch_size)

array([0.61431324, 0.58914906, 0.6314239 ], dtype=float32)

# 3.舆情数据测试

## 3.1 选定文件舆情打分展示

In [11]:
### 定义函数并转换，使得5个库的标准更统一
### score01 代表包2.1得出的情绪分数
### time cost about 3min 54.6s for 119891 sentenses
start_time = time.time()
opinion_df = pd.read_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv')
sentense01 = opinion_df.sentense.astype(str).tolist()
result01 = c.predict(sentense01)
score01 = np.array(result01)[:,1]
opinion_df['score01'] = score01
opinion_df.head()
opinion_df.to_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv',
                  index=False,
                  encoding='utf_8_sig')
opinion_df.head()

,last_change,stock_code,sentense,score01
0,20180406 23:52,600000,浦发为什么这么烂？,0.000478
1,20180406 23:40,600000,反盗版,0.090015
2,20180406 23:29,600000,最近十年搬起石头砸自己的脚”的有: 美国、日本,0.940348
3,20180406 23:21,600000,88见，坐等数票子～,0.964594
4,20180406 23:17,600000,请问这样的股市正常吗？,0.001732


In [12]:
### time cost about 142min for 119891 sentenses
start_time = time.time()
opinion_df = pd.read_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv')
result02 = classifier(sentense01)
score02 = list()
for i in result02:
    if i['label']=='NEGATIVE':
        score_temp = 1-i['score']
    if i['label']=='POSITIVE':
        score_temp = i['score']
    score02.append(score_temp)
#print(score02)
opinion_df['score02'] = score02
opinion_df.head()
opinion_df.to_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv',
                  index=False,
                  encoding='utf_8_sig')
print('time cost is:' ,time.time()-start_time)
opinion_df.head()

time cost is: 2919.618041753769


,last_change,stock_code,sentense,score01,score02
0,20180406 23:52,600000,浦发为什么这么烂？,0.000478,0.040426
1,20180406 23:40,600000,反盗版,0.090015,0.795511
2,20180406 23:29,600000,最近十年搬起石头砸自己的脚”的有: 美国、日本,0.940348,0.327379
3,20180406 23:21,600000,88见，坐等数票子～,0.964594,0.664553
4,20180406 23:17,600000,请问这样的股市正常吗？,0.001732,0.032508


In [13]:
### time cost about 7.1s for 119891 sentenses
start_time = time.time()
opinion_df = pd.read_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv')
def gen03(x):
    import jiagu
    result = jiagu.sentiment(x)
    if result[0]=='negative':
        return 1-result[1]
    else:
        return result[1]
opinion_df['score03'] = opinion_df.sentense.astype(str).apply(lambda x: gen03(x))
opinion_df.to_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv',
                  index=False,
                  encoding='utf_8_sig')
print('time cost is:' ,time.time()-start_time)
opinion_df.head()

time cost is: 2.643435478210449


,last_change,stock_code,sentense,score01,score02,score03
0,20180406 23:52,600000,浦发为什么这么烂？,0.000478,0.040426,0.074351
1,20180406 23:40,600000,反盗版,0.090015,0.795511,0.306777
2,20180406 23:29,600000,最近十年搬起石头砸自己的脚”的有: 美国、日本,0.940348,0.327379,0.001223
3,20180406 23:21,600000,88见，坐等数票子～,0.964594,0.664553,0.120369
4,20180406 23:17,600000,请问这样的股市正常吗？,0.001732,0.032508,0.608623


In [14]:
### time cost about 8min32s for 119891 sentenses
start_time = time.time()
opinion_df = pd.read_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv')
def gen04(x):
    from cnsenti import Sentiment
    senti = Sentiment()
    result = senti.sentiment_count(x)
    pos_score = result['pos']
    neg_score = result['neg']
    if pos_score+neg_score==0:
        return 0.5
    return 0.5 + 0.5*(pos_score/(pos_score+neg_score)) - 0.5*(neg_score/(pos_score+neg_score))
opinion_df['score04'] = opinion_df.sentense.astype(str).apply(lambda x: gen04(x))
opinion_df.head()
opinion_df.to_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv',
                  index=False,
                  encoding='utf_8_sig')
print('time cost is:' ,time.time()-start_time)
opinion_df.head()

time cost is: 163.80170392990112


,last_change,stock_code,sentense,score01,score02,score03,score04
0,20180406 23:52,600000,浦发为什么这么烂？,0.000478,0.040426,0.074351,0.0
1,20180406 23:40,600000,反盗版,0.090015,0.795511,0.306777,0.5
2,20180406 23:29,600000,最近十年搬起石头砸自己的脚”的有: 美国、日本,0.940348,0.327379,0.001223,1.0
3,20180406 23:21,600000,88见，坐等数票子～,0.964594,0.664553,0.120369,0.5
4,20180406 23:17,600000,请问这样的股市正常吗？,0.001732,0.032508,0.608623,1.0


In [15]:
### cost so much time we do it on temp.py file
### time cost about 400min for 119891 sentenses
start_time = time.time()
opinion_df = pd.read_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv')

from utils_selfdefined import predict_score
import paddlenlp as ppnlp
from paddlenlp.datasets import load_dataset
from utils_selfdefined import predict

train_ds, dev_ds, test_ds = load_dataset(
    "chnsenticorp", splits=["train", "dev", "test"])

MODEL_NAME = "ernie-1.0"
tokenizer = ppnlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)
ernie_model = ppnlp.transformers.ErnieModel.from_pretrained(MODEL_NAME)
ernie_model = ppnlp.transformers.ErnieModel.from_pretrained(MODEL_NAME)
model = ppnlp.transformers.ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=len(train_ds.label_list))

def gen05(x,model,tokenizer):

    batch_size = 32
    label_map = {0: 'negative', 1: 'positive'}
        
    temp = predict_score(model, [{"text":x}], tokenizer, label_map, batch_size=batch_size)
    return temp[0]

opinion_df['score05'] = opinion_df.sentense.astype(str).apply(lambda x: gen05(x,model,tokenizer))
opinion_df.to_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv',
                  index=False,
                  encoding='utf_8_sig')
print('time cost is:' ,time.time()-start_time)
opinion_df.head()

[2022-04-15 18:17:23,049] [    INFO] - Already cached /home/cqintern08/.paddlenlp/models/ernie-1.0/vocab.txt
[2022-04-15 18:17:23,060] [    INFO] - Already cached /home/cqintern08/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-04-15 18:17:28,085] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']
[2022-04-15 18:17:28,384] [    INFO] - Already cached /home/cqintern08/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-04-15 18:17:33,318] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']
[2022-04-15 18:17:33,643] [    INFO] - Already cached /home/cqintern08/.paddlenlp/models/ernie-1.0/erni

time cost is: 7045.260102748871


,last_change,stock_code,sentense,score01,score02,score03,score04,score05
0,20180406 23:52,600000,浦发为什么这么烂？,0.000478,0.040426,0.074351,0.0,0.488139
1,20180406 23:40,600000,反盗版,0.090015,0.795511,0.306777,0.5,0.461863
2,20180406 23:29,600000,最近十年搬起石头砸自己的脚”的有: 美国、日本,0.940348,0.327379,0.001223,1.0,0.482783
3,20180406 23:21,600000,88见，坐等数票子～,0.964594,0.664553,0.120369,0.5,0.423272
4,20180406 23:17,600000,请问这样的股市正常吗？,0.001732,0.032508,0.608623,1.0,0.390910


In [ ]:
### 之前数据的读取存储出现了编码错误，该cell用于校正该错误，正常情况下无用，若是误运行，不会对文件产生影响。
opinion_df = pd.read_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv')
opinion_df.to_csv('/home/cqintern08/AbsoluteX_intern/Absolute_Alpha/motion_score_library_test/opinion_df.csv',
                  index=False,
                  encoding='utf_8_sig')